# Метапрограммирование  Декораторы

In [1]:
_sqr_cache = {}
def sqr(x):
    if x not in _sqr_cache:
        _sqr_cache[x] = x * x
    return _sqr_cache[x]

half_cache = {}
def half(x):
    if x not in _half_cache:
        _half_cache[x] = x // 2
    return _half_cache[x]


In [2]:
sqr(5),sqr(20),_sqr_cache


(25, 400, {5: 25, 20: 400})

In [33]:
def cached(func):
    cache= {}
    def new_func(x):
        if x not in cache:
            cache[x] = func(x)
        return cache[x]
    return new_func

@cached
def sqr_c(x):
    print("cached function sqr_c")
    return x*x
@cached
def half_c(x):
    print("cached function half")
    return x // 2


In [34]:
sqr_c(5),sqr_c(5),half_c(10),half_c(10)

cached function sqr_c
cached function half


(25, 25, 5, 5)

## Декоратор функции с параметрами

In [63]:
from functools import wraps

def cached_l(limit):
    def decorator(func):
        cache= {}
        #wraps - "проброс" описания функции sqr_l.__doc__
        @wraps(func)           
        def new_func(x):
            if len(cache)>=limit:
                print(len(cache)," ",limit)
                raise RuntimeError
                cache.clear()
                
            if x not in cache:
                cache[x] = func(x)
            return cache[x]
        return new_func
    return decorator

@cached_l(limit=3)
def sqr_l(x):
    """cached function sqr_l """
    print("cached function sqr_l ", x)
    return x*x


## Декораторы класса

In [12]:
def classcached(method):
    method_name = method.__name__
    print(method)
    
    def new_method(self):
        if method_name not in self.CACHE:
            self.CACHE[method_name] = {}
        cache = self.CACHE[method_name]
        
        if self._value not in cache:
            cache[self._value] = method(self)
        return cache[self._value]
    return new_method

class Number:
    CACHE = {}
    
    def __init__(self,value):
        self._value = value
    
    @classcached
    def sqr(self):
        return type(self)(self._value * self._value)
    
    @classcached    
    def half(self):
        return type(self)(self._value // 2)
    
    def __repr__(self):
        return 'Number({0})'.format(self._value)
    



<function Number.sqr at 0x04A62780>
<function Number.half at 0x04A62AE0>


In [13]:
a = Number(4).sqr()

In [14]:
a.CACHE


{'sqr': {4: Number(16)}}

## Кеширование всех методов класса

In [23]:
def class_all_cached(method):
    method_name = method.__name__
    print(method)
    
    def new_method(self):
        if method_name not in self.CACHE:
            self.CACHE[method_name] = {}
        cache = self.CACHE[method_name]
        
        if self._value not in cache:
            cache[self._value] = method(self)
        return cache[self._value]
    return new_method

def cache_all(cclass):
    for attr_name in ['sqr','half']:         ### ???
        attr = getattr(cclass,attr_name)
        setattr(cclass,attr_name,class_all_cached(attr))
    return cclass

@cache_all
class NumberAll:
    CACHE = {}
    
    def __init__(self,value):
        self._value = value
    
    def sqr(self):
        return type(self)(self._value * self._value)
    
    def half(self):
        return type(self)(self._value // 2)
    
    def __repr__(self):
        return 'Number({0})'.format(self._value)
    


<function NumberAll.sqr at 0x04A62858>
<function NumberAll.half at 0x04A62ED0>


### Автоматическое кеширование отмеченных методов класса

In [39]:
def cachedAuto(method):
    method._cached = True
    return method


def make_cached(method,cache_name):
    method_name = method.__name__
    print(method)
    
    def new_method(self):
        global_cache = getattr(self,cache_name)
        if method_name not in global_cache:
            global_cache[method_name] = {}
        cache = global_cache[method_name]
        
        if self._value not in cache:
            cache[self._value] = method(self)
        return cache[self._value]
    return new_method

#def cached_auto(cclass):    
#    for attr_name in cclass.__dict__:
#        attr = getattr(cclass,attr_name)
#        if hasattr(attr,'_cached') and attr._cached:
#            setattr(cclass,attr_name,make_cached(attr))
#    return cclass

def cached_autoM(cache_name):
    def decorator(cclass):
        setattr(cclass,cache_name,{})
        
        for attr_name in cclass.__dict__:
            attr = getattr(cclass,attr_name)
            if hasattr(attr,'_cached') and attr._cached:
                setattr(cclass,attr_name,make_cached(attr,cache_name))
        return cclass
    return decorator


@cached_autoM('CACHE_M')
class NumberA:
    ###CACHE = {}
    
    def __init__(self,value):
        self._value = value
    
    @cachedAuto
    def sqr(self):
        return type(self)(self._value * self._value)
    
    @cachedAuto
    def half(self):
        return type(self)(self._value // 2)
    
    def __repr__(self):
        return 'Number({0})'.format(self._value)
    


<function NumberA.sqr at 0x04A6E030>
<function NumberA.half at 0x04A6E6A8>


In [40]:
NumberA(3).sqr(),NumberA.CACHE_M

(Number(9), {'sqr': {3: Number(9)}})

In [41]:
NumberA(3).sqr()


Number(9)